In [2]:
# importing torch/torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# MNIST image size: 28x28 = 784
# MNIST num classes: 10 (digits 0-9)
model = NN(784, 10)
x = torch.randn(64, 784)

In [3]:
# verify
model(x).shape

torch.Size([64, 10])

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # run on gpu (if there)

In [3]:
# hyperparams
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [4]:
# load data
# downloads MNIST if not already downloaded
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
# initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [8]:
# loss + optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # reshaping data
        data = data.reshape(data.shape[0], -1)

        # forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # backward pass
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

In [10]:
# check accuracy on training/test data
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data.')
    else:
        print('Checking accuracy on test data.')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1) # gives us the prediction (digit w/ maximum likelihood)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

In [11]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data.
Got 59022 / 60000 with accuracy 98.37
Checking accuracy on test data.
Got 9718 / 10000 with accuracy 97.18


In [20]:
# making a CNN
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # run on gpu (if there)

In [30]:
# hyperparams
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 15

In [31]:
model = CNN(in_channels = in_channel, num_classes = num_classes).to(device)

In [32]:
# loss + optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# load data
# downloads MNIST if not already downloaded
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [34]:
test_dataset[0][0].shape

torch.Size([1, 28, 28])

In [35]:
# train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # backward pass
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

In [36]:
# check accuracy on training/test data
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on training data.')
    else:
        print('Checking accuracy on test data.')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1) # gives us the prediction (digit w/ maximum likelihood)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

In [37]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data.
Got 59674 / 60000 with accuracy 99.46
Checking accuracy on test data.
Got 9877 / 10000 with accuracy 98.77


In [42]:
FILEPATH = "mnist_model.pt"
torch.save(model.state_dict(), FILEPATH)

In [43]:
FILEPATH2 = "mnist_model.pb"
torch.save(model, FILEPATH2)